In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, platform, warnings
import datasets;
from datasets import load_dataset, Dataset, DatasetDict
;
from trl import SFTTrainer
from huggingface_hub import notebook_login
#Use a sharded model to fine-tune in the free version of Google Colab.
# base_model = "mistralai/Mistral-7B-v0.1"
model_name = "Orkhan/llama-2-7b-absa"

2024-08-09 12:09:19.699375: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 12:09:19.699428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 12:09:19.700935: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# del model
model = None
import gc
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
# LOAD MODEL QUANTİZE
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

###### if you load llama-2-7b original weights, use the following 2 lines to align your model's weights to original model's weights
model = PeftModel.from_pretrained(base_model, "/kaggle/working/absa_training_model")
model = model.merge_and_unload()

# tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"])
# if you are quantizing the model use  model else base_model
model = get_peft_model(model, peft_config)

In [ ]:
data=[  
    '<s>[INST] ### Human: Vodafone müşteri destek hattı çok nazikti, fakat çağrı süresi biraz uzun sürdü. ### [/INST] ### Assistant: ## Aspect detected: Vodafone müşteri destek hattı, çağrı süresi ## Opinion detected: nazik, uzun sürdü ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un interneti çok hızlı, ancak müşteri hizmetleriyle iletişim zordu. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un interneti, müşteri hizmetleri ## Opinion detected: hızlı, zordu ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in mobil veri paketi oldukça uygun fiyatlı, ama kapsama alanı bazı bölgelerde zayıf. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in mobil veri paketi, kapsama alanı ## Opinion detected: uygun fiyatlı, zayıf ## Sentiment detected: Positive, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Vodafone\'un teknik destek servisi çok profesyoneldi, fakat internet kesintileri sık yaşanıyor. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un teknik destek servisi, internet kesintileri ## Opinion detected: profesyonel, sık yaşanıyor ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un fiber interneti harika çalışıyor, ancak kurulum süreci oldukça uzun sürdü. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un fiber interneti, kurulum süreci ## Opinion detected: harika çalışıyor, uzun sürdü ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in müşteri hizmetleri son derece yardımcıydı, ama ürün teslimat süresi uzun. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in müşteri hizmetleri, ürün teslimat süresi ## Opinion detected: yardımcı, uzun ## Sentiment detected: Positive, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Vodafone\'un yeni telefon kampanyası çok cazip, fakat satış sonrası destek yetersiz. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un yeni telefon kampanyası, satış sonrası destek ## Opinion detected: cazip, yetersiz ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un 4.5G hizmeti oldukça hızlı, fakat fiyatlar biraz yüksek. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un 4.5G hizmeti, fiyatlar ## Opinion detected: hızlı, yüksek ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in müşteri destek ekibi gerçekten etkili, ancak internet hızım tatmin edici değil. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in müşteri destek ekibi, internet hızı ## Opinion detected: etkili, tatmin edici değil ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un yeni tarifesi oldukça uygun, ama internet bağlantı sorunları yaşıyorum. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un yeni tarifesi, internet bağlantı sorunları ## Opinion detected: uygun, yaşıyorum ## Sentiment detected: Positive, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Türk Telekom\'un müşteri hizmetleri çok yardımcıydı, ancak Vodafone\'un internet hızım beklediğimden düşük. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un müşteri hizmetleri, Vodafone\'un internet hızı ## Opinion detected: yardımcı, düşük ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in müşteri desteği oldukça profesyoneldi, ama Türk Telekom\'un çağrı süresi çok uzun sürdü. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in müşteri desteği, Türk Telekom\'un çağrı süresi ## Opinion detected: profesyonel, uzun sürdü ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un yeni paketleri cazip, ancak Türkcell\'in internet bağlantı sorunları yaşatıyor. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un yeni paketleri, Türkcell\'in internet bağlantı sorunları ## Opinion detected: cazip, yaşatıyor ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un fiber interneti harika, fakat Türkcell\'in müşteri hizmetleri yetersiz. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un fiber interneti, Türkcell\'in müşteri hizmetleri ## Opinion detected: harika, yetersiz ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in mobil verisi uygun fiyatlı, fakat Vodafone\'un kurulum süreci çok uzun sürdü. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in mobil verisi, Vodafone\'un kurulum süreci ## Opinion detected: uygun fiyatlı, uzun sürdü ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un 4.5G hizmeti çok hızlı, ama Türkcell\'in fiyatları oldukça yüksek. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un 4.5G hizmeti, Türkcell\'in fiyatları ## Opinion detected: hızlı, yüksek ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un çağrı merkezi çok hızlı yanıt veriyor, fakat Türk Telekom\'un veri hızı beklediğimden düşük. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un çağrı merkezi, Türk Telekom\'un veri hızı ## Opinion detected: hızlı yanıt veriyor, düşük ## Sentiment detected: Positive, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Türkcell\'in kampanya teklifleri çok cazip, ama Vodafone\'un müşteri desteği yetersiz. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in kampanya teklifleri, Vodafone\'un müşteri desteği ## Opinion detected: cazip, yetersiz ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un internet paketi uygun fiyatlı, fakat Türkcell\'in çağrı süresi çok uzun. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un internet paketi, Türkcell\'in çağrı süresi ## Opinion detected: uygun fiyatlı, uzun ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un teknik destek servisi çok etkili, ama Türk Telekom\'un bağlantı hızında sık sık sorun yaşıyorum. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un teknik destek servisi, Türk Telekom\'un bağlantı hızı ## Opinion detected: etkili, sık sık sorun yaşıyorum ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un müşteri hizmetleri ve Vodafone\'un internet hızını beğeniyorum, ancak Türkcell\'in veri planları yüksek fiyatlı. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un müşteri hizmetleri, Vodafone\'un internet hızı, Türkcell\'in veri planları ## Opinion detected: beğeniyorum, yüksek fiyatlı ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türkcell\'in yeni akıllı telefonları oldukça cazip, ama Türk Telekom\'un internet bağlantı süresi uzun ve Vodafone\'un teknik desteği yetersiz. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in yeni akıllı telefonları, Türk Telekom\'un internet bağlantı süresi, Vodafone\'un teknik desteği ## Opinion detected: cazip, uzun, yetersiz ## Sentiment detected: Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un müşteri desteği çok hızlı, Türkcell\'in mobil veri paketi uygun fiyatlı, fakat Türk Telekom\'un çağrı merkezi bazı sorunlar yaşatıyor. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un müşteri desteği, Türkcell\'in mobil veri paketi, Türk Telekom\'un çağrı merkezi ## Opinion detected: hızlı, uygun fiyatlı, bazı sorunlar yaşatıyor ## Sentiment detected: Positive, Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un fiber interneti çok iyi çalışıyor ve Türkcell\'in kampanya fiyatları cazip, fakat Vodafone\'un müşteri hizmetleri beklediğim gibi değil. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un fiber interneti, Türkcell\'in kampanya fiyatları, Vodafone\'un müşteri hizmetleri ## Opinion detected: iyi çalışıyor, cazip, beklediğim gibi değil ## Sentiment detected: Positive, Positive, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Türkcell\'in 4.5G hizmeti harika, fakat Türk Telekom\'un kurulum süresi uzun ve Vodafone\'un internet hızı bazen yavaşlıyor. ### [/INST] ### Assistant: ## Aspect detected: Türkcell\'in 4.5G hizmeti, Türk Telekom\'un kurulum süresi, Vodafone\'un internet hızı ## Opinion detected: harika, uzun, bazen yavaşlıyor ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türk Telekom\'un yeni veri paketi uygun fiyatlı, Türkcell\'in müşteri hizmetleri çok etkili ve Vodafone\'un çağrı süresi biraz uzun. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un yeni veri paketi, Türkcell\'in müşteri hizmetleri, Vodafone\'un çağrı süresi ## Opinion detected: uygun fiyatlı, etkili, uzun ## Sentiment detected: Positive, Positive, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Vodafone\'un teknik destek servisi çok profesyonel, ama Türkcell\'in internet hızında sık sık kesintiler oluyor ve Türk Telekom\'un fiyatları yüksek. ### [/INST] ### Assistant: ## Aspect detected: Vodafone\'un teknik destek servisi, Türkcell\'in internet hızı, Türk Telekom\'un fiyatları ## Opinion detected: profesyonel, sık sık kesintiler, yüksek ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Apple\'ın yeni iPhone modeli çok etkileyici, ancak Samsung\'un müşteri desteği biraz yetersiz ve Sony\'nin kulaklıklarının sesi düşük. ### [/INST] ### Assistant: ## Aspect detected: Apple\'ın yeni iPhone modeli, Samsung\'un müşteri desteği, Sony\'nin kulaklıkları ## Opinion detected: etkileyici, yetersiz, sesi düşük ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Tesla\'nın elektrikli araçları oldukça hızlı, fakat BMW\'nin servis süreci uzun ve Audi\'nin fiyatları yüksek. ### [/INST] ### Assistant: ## Aspect detected: Tesla\'nın elektrikli araçları, BMW\'nin servis süreci, Audi\'nin fiyatları ## Opinion detected: hızlı, uzun, yüksek ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Microsoft\'un Surface Pro cihazları çok kullanışlı, ancak Lenovo\'nun teknik desteği bazı sorunlar yaşatıyor ve HP\'nin laptoplarının bataryası kısa ömürlü. ### [/INST] ### Assistant: ## Aspect detected: Microsoft\'un Surface Pro cihazları, Lenovo\'nun teknik desteği, HP\'nin laptopları ## Opinion detected: kullanışlı, bazı sorunlar yaşatıyor, kısa ömürlü ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: LG\'nin OLED televizyonları mükemmel bir görüntü kalitesine sahip, ama Samsung\'un ses sistemleri bazen bozuluyor ve Sony\'nin müşteri hizmetleri yetersiz. ### [/INST] ### Assistant: ## Aspect detected: LG\'nin OLED televizyonları, Samsung\'un ses sistemleri, Sony\'nin müşteri hizmetleri ## Opinion detected: mükemmel, bazen bozuluyor, yetersiz ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Huawei\'nin akıllı saatleri çok kullanışlı, fakat Fitbit\'in kalp rate sensörü bazen doğru ölçmüyor ve Garmin\'in GPS cihazlarının bataryası çabuk bitiyor. ### [/INST] ### Assistant: ## Aspect detected: Huawei\'nin akıllı saatleri, Fitbit\'in kalp rate sensörü, Garmin\'in GPS cihazları ## Opinion detected: kullanışlı, bazen doğru ölçmüyor, çabuk bitiyor ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Nike\'ın spor ayakkabıları oldukça konforlu, ancak Adidas\'ın ürünleri fiyat olarak yüksek ve Puma\'nın teslimat süresi uzun. ### [/INST] ### Assistant: ## Aspect detected: Nike\'ın spor ayakkabıları, Adidas\'ın ürünleri, Puma\'nın teslimat süresi ## Opinion detected: konforlu, yüksek, uzun ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Canon\'un DSLR kameraları mükemmel fotoğraf kalitesi sunuyor, ama Nikon\'un lensleri çok ağır ve Sony\'nin video performansı yeterli değil. ### [/INST] ### Assistant: ## Aspect detected: Canon\'un DSLR kameraları, Nikon\'un lensleri, Sony\'nin video performansı ## Opinion detected: mükemmel, çok ağır, yeterli değil ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Dell\'in bilgisayarları hızlı ve güvenilir, fakat Acer\'ın ekran çözünürlüğü düşük ve Asus\'un müşteri hizmetleri biraz yetersiz. ### [/INST] ### Assistant: ## Aspect detected: Dell\'in bilgisayarları, Acer\'ın ekran çözünürlüğü, Asus\'un müşteri hizmetleri ## Opinion detected: hızlı ve güvenilir, düşük, biraz yetersiz ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Philips\'in elektrikli diş fırçası çok etkili, ama Oral-B\'nin batarya ömrü kısa ve Colgate\'nin diş macunu biraz sert. ### [/INST] ### Assistant: ## Aspect detected: Philips\'in elektrikli diş fırçası, Oral-B\'nin batarya ömrü, Colgate\'nin diş macunu ## Opinion detected: etkili, kısa, biraz sert ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Bose\'un gürültü engelleyici kulaklıkları harika, fakat Beats\'in ses kalitesi bazen bozuk ve Sennheiser\'ın tasarımı biraz eski moda. ### [/INST] ### Assistant: ## Aspect detected: Bose\'un gürültü engelleyici kulaklıkları, Beats\'in ses kalitesi, Sennheiser\'ın tasarımı ## Opinion detected: harika, bazen bozuk, biraz eski moda ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Superonline\'ın internet hızı mükemmel, ancak Türksat\'ın müşteri hizmetleri biraz yetersiz ve Avea\'nın veri planları yüksek fiyatlı. ### [/INST] ### Assistant: ## Aspect detected: Superonline\'ın internet hızı, Türksat\'ın müşteri hizmetleri, Avea\'nın veri planları ## Opinion detected: mükemmel, biraz yetersiz, yüksek fiyatlı ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: BiP uygulamasının arayüzü çok kullanışlı, fakat Türk Telekom\'un fiber internetinde sık sık kesintiler yaşanıyor ve Türksat\'ın teknik desteği bazen yetersiz. ### [/INST] ### Assistant: ## Aspect detected: BiP uygulamasının arayüzü, Türk Telekom\'un fiber interneti, Türksat\'ın teknik desteği ## Opinion detected: kullanışlı, sık sık kesintiler, bazen yetersiz ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Superonline\'ın yeni kampanya teklifleri cazip, ancak Türksat\'ın internet bağlantı süresi uzun ve Avea\'nın müşteri desteği etkisiz. ### [/INST] ### Assistant: ## Aspect detected: Superonline\'ın yeni kampanya teklifleri, Türksat\'ın internet bağlantı süresi, Avea\'nın müşteri desteği ## Opinion detected: cazip, uzun, etkisiz ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: BiP uygulaması oldukça hızlı, ama Superonline\'ın internet hızında bazı sorunlar yaşanıyor ve Türksat\'ın kurulum süreci çok uzun. ### [/INST] ### Assistant: ## Aspect detected: BiP uygulaması, Superonline\'ın internet hızı, Türksat\'ın kurulum süreci ## Opinion detected: hızlı, bazı sorunlar yaşanıyor, çok uzun ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Türk Telekom\'un çağrı merkezi iyi çalışıyor, ancak Avea\'nın veri hızında sık sık kesintiler yaşanıyor ve Superonline\'ın fiyatları yüksek. ### [/INST] ### Assistant: ## Aspect detected: Türk Telekom\'un çağrı merkezi, Avea\'nın veri hızı, Superonline\'ın fiyatları ## Opinion detected: iyi çalışıyor, sık sık kesintiler, yüksek ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Superonline\'ın fiber interneti çok hızlı, ama Türksat\'ın müşteri desteği oldukça yetersiz ve Avea\'nın internet paketi çok pahalı. ### [/INST] ### Assistant: ## Aspect detected: Superonline\'ın fiber interneti, Türksat\'ın müşteri desteği, Avea\'nın internet paketi ## Opinion detected: çok hızlı, oldukça yetersiz, çok pahalı ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: BiP\'in sesli görüşme kalitesi mükemmel, fakat Avea\'nın veri planları pahalı ve Superonline\'ın kurulum süresi uzun. ### [/INST] ### Assistant: ## Aspect detected: BiP\'in sesli görüşme kalitesi, Avea\'nın veri planları, Superonline\'ın kurulum süresi ## Opinion detected: mükemmel, pahalı, uzun ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Türksat\'ın uydu hizmetleri çok iyi, ancak Superonline\'ın müşteri desteği yetersiz ve Avea\'nın internet hızında bazı sorunlar var. ### [/INST] ### Assistant: ## Aspect detected: Türksat\'ın uydu hizmetleri, Superonline\'ın müşteri desteği, Avea\'nın internet hızı ## Opinion detected: çok iyi, yetersiz, bazı sorunlar var ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: BiP uygulamasının grup sohbetleri oldukça kullanışlı, ama Superonline\'ın interneti bazı zamanlarda yavaşlıyor ve Avea\'nın müşteri hizmetleri eksik kalıyor. ### [/INST] ### Assistant: ## Aspect detected: BiP uygulamasının grup sohbetleri, Superonline\'ın interneti, Avea\'nın müşteri hizmetleri ## Opinion detected: kullanışlı, bazı zamanlarda yavaşlıyor, eksik kalıyor ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Superonline\'ın 5G hizmeti harika, fakat Türksat\'ın teknik desteği yeterince hızlı değil ve Avea\'nın veri paketleri sık sık doluyor. ### [/INST] ### Assistant: ## Aspect detected: Superonline\'ın 5G hizmeti, Türksat\'ın teknik desteği, Avea\'nın veri paketleri ## Opinion detected: harika, yeterince hızlı değil, sık sık doluyor ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Kick Turkey\'nin düzenlediği spor etkinliği çok canlıydı, fakat Garanti BBVA\'nın finansal teknoloji forumu biraz zayıftı ve Turkcell\'in dijital dönüşüm çalıştayı yeterince ilgi çekmedi. ### [/INST] ### Assistant: ## Aspect detected: Kick Turkey\'nin spor etkinliği, Garanti BBVA\'nın finansal teknoloji forumu, Turkcell\'in dijital dönüşüm çalıştayı ## Opinion detected: çok canlıydı, biraz zayıftı, yeterince ilgi çekmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Akbank\'ın düzenlediği girişimcilik zirvesi oldukça ilgi çekiciydi, ancak Yapı Kredi\'nin fintech semineri daha kapsamlı olabilirdi ve Ziraat Bankası\'nın tarım teknolojileri etkinliği beklenen ilgiyi görmedi. ### [/INST] ### Assistant: ## Aspect detected: Akbank\'ın girişimcilik zirvesi, Yapı Kredi\'nin fintech semineri, Ziraat Bankası\'nın tarım teknolojileri etkinliği ## Opinion detected: oldukça ilgi çekiciydi, daha kapsamlı olabilirdi, beklenen ilgiyi görmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>'
    '<s>[INST] ### Human: Anadolu Efes\'in organize ettiği spor turnuvası çok heyecanlıydı, fakat Coca-Cola\'nın pazarlama konferansı daha etkileşimli olabilirdi ve Ülker\'in gıda inovasyon etkinliği fazla teknikti. ### [/INST] ### Assistant: ## Aspect detected: Anadolu Efes\'in spor turnuvası, Coca-Cola\'nın pazarlama konferansı, Ülker\'in gıda inovasyon etkinliği ## Opinion detected: çok heyecanlıydı, daha etkileşimli olabilirdi, fazla teknikti ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: PepsiCo\'nun düzenlediği tüketici etkinliği oldukça ilgi çekiciydi, ancak Beko\'nun ev teknolojileri sergisi daha yenilikçi olabilirdi ve Bosch\'un mühendislik semineri biraz sıkıcıydı. ### [/INST] ### Assistant: ## Aspect detected: PepsiCo\'nun tüketici etkinliği, Beko\'nun ev teknolojileri sergisi, Bosch\'un mühendislik semineri ## Opinion detected: oldukça ilgi çekiciydi, daha yenilikçi olabilirdi, biraz sıkıcıydı ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Ford\'un otomobil tanıtım etkinliği çok başarılıydı, fakat Mercedes-Benz\'in teknoloji çalıştayı daha etkileşimli olabilirdi ve Toyota\'nın sürdürülebilirlik semineri daha geniş bir kitleye hitap edebilirdi. ### [/INST] ### Assistant: ## Aspect detected: Ford\'un otomobil tanıtım etkinliği, Mercedes-Benz\'in teknoloji çalıştayı, Toyota\'nın sürdürülebilirlik semineri ## Opinion detected: çok başarılıydı, daha etkileşimli olabilirdi, daha geniş bir kitleye hitap edebilirdi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: Trendyol\'un e-ticaret etkinliği oldukça verimli geçti, fakat Hepsiburada\'nın lojistik forumu daha kapsamlı olabilirdi ve GittiGidiyor\'un müşteri memnuniyeti günleri beklenen ilgiyi görmedi. ### [/INST] ### Assistant: ## Aspect detected: Trendyol\'un e-ticaret etkinliği, Hepsiburada\'nın lojistik forumu, GittiGidiyor\'un müşteri memnuniyeti günleri ## Opinion detected: oldukça verimli geçti, daha kapsamlı olabilirdi, beklenen ilgiyi görmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @TurkTelekom\'un düzenlediği #TeknolojiFuarı gerçekten etkileyiciydi, ama @VodafoneTR\'in #MüşteriHizmetleri beklentilerimi karşılamadı ve @Turkcell\'in #VeriPlanları biraz pahalı. ### [/INST] ### Assistant: ## Aspect detected: @TurkTelekom\'un Teknoloji Fuarı, @VodafoneTR\'in Müşteri Hizmetleri, @Turkcell\'in Veri Planları ## Opinion detected: gerçekten etkileyiciydi, beklentilerimi karşılamadı, biraz pahalı ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @GarantiBBVA\'nın #FinansSemineri çok bilgilendiriciydi, fakat @YapiKredi\'nin #FintechEtkinliği biraz eksikti ve @IsBankasi\'nın #MüşteriHizmetleriGünleri yeterince ilgi görmedi. ### [/INST] ### Assistant: ## Aspect detected: @GarantiBBVA\'nın Finans Semineri, @YapiKredi\'nin Fintech Etkinliği, @IsBankasi\'nın Müşteri Hizmetleri Günleri ## Opinion detected: çok bilgilendiriciydi, biraz eksikti, yeterince ilgi görmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>'
    '<s>[INST] ### Human: @Trendyol\'un #E-TicaretEtkinliği oldukça verimli geçti, ancak @Hepsiburada\'nın #LojistikForum\'u daha kapsamlı olabilirdi ve @GittiGidiyor\'un #MüşteriMemnuniyetiGünleri ilgi çekmedi. ### [/INST] ### Assistant: ## Aspect detected: @Trendyol\'un E-Ticaret Etkinliği, @Hepsiburada\'nın Lojistik Forumu, @GittiGidiyor\'un Müşteri Memnuniyeti Günleri ## Opinion detected: oldukça verimli geçti, daha kapsamlı olabilirdi, ilgi çekmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @Ford\'un #OtomobilLansmanı çok başarılıydı, fakat @MercedesBenz\'in #TeknolojiÇalıştayı daha etkileşimli olabilirdi ve @Toyota\'nın #SürdürülebilirlikSemineri daha geniş bir kitleye hitap edebilirdi. ### [/INST] ### Assistant: ## Aspect detected: @Ford\'un Otomobil Lansmanı, @MercedesBenz\'in Teknoloji Çalıştayı, @Toyota\'nın Sürdürülebilirlik Semineri ## Opinion detected: çok başarılıydı, daha etkileşimli olabilirdi, daha geniş bir kitleye hitap edebilirdi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @PepsiCo\'nun #TüketiciEtkinliği oldukça ilgi çekiciydi, fakat @Beko\'nun #EvTeknolojileriSergisi daha yenilikçi olabilirdi ve @Bosch\'un #MühendislikSemineri biraz sıkıcıydı. ### [/INST] ### Assistant: ## Aspect detected: @PepsiCo\'nun Tüketici Etkinliği, @Beko\'nun Ev Teknolojileri Sergisi, @Bosch\'un Mühendislik Semineri ## Opinion detected: oldukça ilgi çekiciydi, daha yenilikçi olabilirdi, biraz sıkıcıydı ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @KickTurkey\'nin #SporEtkinliği çok canlıydı, ancak @GarantiBBVA\'nın #FinansTeknolojiForum\'u biraz zayıftı ve @Turkcell\'in #DijitalDönüşümÇalıştayı yeterince ilgi çekmedi. ### [/INST] ### Assistant: ## Aspect detected: @KickTurkey\'nin Spor Etkinliği, @GarantiBBVA\'nın Finans Teknoloji Forumu, @Turkcell\'in Dijital Dönüşüm Çalıştayı ## Opinion detected: çok canlıydı, biraz zayıftı, yeterince ilgi çekmedi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>',
    '<s>[INST] ### Human: @TetraPak\'ın #GıdaTeknolojileriEtkinliği harikaydı, fakat @Unilever\'in #SürdürülebilirlikSemineri daha etkileyici olabilirdi ve @Nestle\'in #SağlıkGünü biraz yetersizdi. ### [/INST] ### Assistant: ## Aspect detected: @TetraPak\'ın Gıda Teknolojileri Etkinliği, @Unilever\'in Sürdürülebilirlik Semineri, @Nestle\'in Sağlık Günü ## Opinion detected: harikaydı, daha etkileyici olabilirdi, biraz yetersizdi ## Sentiment detected: Positive, Negative, Negative)\n\n##</s>'  
]

In [ ]:
df=pd.DataFrame(data,columns=["prompts"])

In [ ]:
df.prompts.values[0]

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.column_names

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Training Arguments
# Hyperparameters should beadjusted based on the hardware you using
training_arguments = TrainingArguments(
    output_dir= "absa_training",
    num_train_epochs= 3,
    per_device_train_batch_size= 1,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 5000,
    logging_steps= 30,
    learning_rate= 5e-6,
    weight_decay= 0.01,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="none",
    push_to_hub=True
)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="prompts",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
# Save the fine-tuned model
model_name = "absa_training_model"
trainer.model.save_pretrained(model_name)
model.config.use_cache = True
model.eval()

In [ ]:
pre_model = PeftModel.from_pretrained(model, "/kaggle/working/absa_training_model")
pre_model = pre_model.merge_and_unload()

In [ ]:
def process_output(result, user_prompt):
    interpreted_input = result[0]['generated_text'].split('### Assistant:')[0].split('### Human:')[1]
    print("interpreted:", interpreted_input)
    new_output = result[0]['generated_text'].split('### Assistant:')[1].split(')')[0].strip()
    print("new out asist:", new_output)

    new_output.split('## Opinion detected:')
    print("New out Opin:", interpreted_input)

    aspect_opinion_sentiment = new_output
    print("AOS:", aspect_opinion_sentiment)

    aspects = aspect_opinion_sentiment.split('Aspect detected:')[1].split('##')[0]
    opinions = aspect_opinion_sentiment.split('Opinion detected:')[1].split('## Sentiment detected:')[0]
    sentiments = aspect_opinion_sentiment.split('## Sentiment detected:')[1]


    aspect_list = [aspect.strip() for aspect in aspects.split(',') if ',' in aspects]
    opinion_list = [opinion.strip() for opinion in opinions.split(',') if ',' in opinions]
    sentiments_list = [sentiment.strip() for sentiment in sentiments.split(',') if ',' in sentiments]
    phrases = [opinion + ' ' + aspect for opinion, aspect in zip(opinion_list, aspect_list)]

    output_dict = {
        'user_prompt': user_prompt,
        'interpreted_input': interpreted_input,
        'aspects': aspect_list,
        'opinions': opinion_list,
        'sentiments': sentiments_list,
        'phrases': phrases
    }

    return output_dict


def process_prompt(user_prompt, model):
    edited_prompt = "### Human: " + user_prompt + '.###'
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=len(tokenizer.encode(user_prompt))*3.5)
    result = pipe(edited_prompt)
    print("*******",result)
    print()
    output_dict = process_output(result, user_prompt)
    return result, output_dict

In [ ]:
prompt = "Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey  gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell"
raw_result, output_dict = process_prompt(prompt, pre_model)

# print(raw_result)

output_dict